In [0]:
dbutils.fs.unmount('/mnt/rawdata/')

/mnt/rawdata/ has been unmounted.
Out[16]: True

In [0]:
dbutils.fs.mount(
  source = "wasbs://iplusecase4files@ipldata1.blob.core.windows.net",
  mount_point = "/mnt/rawdata",
  extra_configs = {"fs.azure.account.key.ipldata1.blob.core.windows.net":'2XSYMNiYfQa46wru4hJbIY8SQbhmEj7mVPXbj3JDSMCMnjf7fPRKijtlFQWReqWO14v8/YbF4d1B+AStGq2bvg=='})

Out[17]: True

In [0]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.functions import lit

In [0]:
spark = SparkSession.builder.appName("Licene Optimization").getOrCreate()

In [0]:
%fs ls dbfs:/mnt/

path,name,size,modificationTime
dbfs:/mnt/rawdata/,rawdata/,0,0


In [0]:
df_ps1 = spark.read.csv("dbfs:/mnt/rawdata/src/IPL_CC_Users.csv", header = True, inferSchema = True)
df_ps2 = spark.read.csv("dbfs:/mnt/rawdata/src/IPL_CC_CN_Users.csv", header = True, inferSchema = True)
df_ps3 = spark.read.csv("dbfs:/mnt/rawdata/src/IPL_DC_Users.csv", header = True, inferSchema = True)
df_ps4 = spark.read.csv("dbfs:/mnt/rawdata/src/IPL_DC_CN_Users.csv", header = True, inferSchema = True)

In [0]:
CC_Rem = df_ps1[df_ps1['Country'] == 'CN']
DC_Rem = df_ps3[df_ps3['Country'] == 'CN'] # Seperating the CN region users present in other grps
CC_CN_Rem = df_ps2[~(df_ps2['Country'] == 'CN')]
DC_CN_Rem = df_ps4[~(df_ps4['Country'] == 'CN')] #Seperating the other region users from CN group

In [0]:
df_ps1 = df_ps1.union(df_ps2)

In [0]:
df_ps3 = df_ps3.union(df_ps4)

In [0]:
del df_ps2,df_ps4

In [0]:
df_ps1.columns

Out[25]: ['Application Access',
 'First name',
 'Last name',
 'Country',
 'Username',
 'Mail',
 'BU',
 'Cost']

In [0]:
dup = df_ps1.join(df_ps3,df_ps1.Mail==df_ps3.Mail,"inner").select(df_ps1.Username,df_ps1.Mail,df_ps1.Country, df_ps1.BU)

In [0]:
dup.count()

Out[27]: 51

In [0]:
duplicates = dup.toPandas()

In [0]:
duplicates.to_csv("/dbfs/mnt/rawdata/dest/Duplicates.csv", index = False)

In [0]:
dup.show(truncate= True)

+-----------------+--------------------+-------+---+
|         Username|                Mail|Country| BU|
+-----------------+--------------------+-------+---+
|  jerrica.cameron|jerrica.cameron@e...|     NO| IT|
|  terrance.moreno|terrance.moreno@e...|     DO| IT|
|     kellee.heath|kellee.heath@emai...|     JP| IT|
|      burl.burton|burl.burton@email...|     SE| IT|
|    ha.livingston|ha.livingston@ema...|     BE| IT|
|    stevie.pierce|stevie.pierce@ema...|     MX| IT|
| christena.willis|christena.willis@...|     VN| IT|
|  dahlia.caldwell|dahlia.caldwell@e...|     VN| IT|
|   alfredo.cherry|alfredo.cherry@em...|     TH| IT|
|   valrie.salinas|valrie.salinas@em...|     AE| IT|
|     eladia.gross|eladia.gross@emai...|     MX| IT|
|mitsue.richardson|mitsue.richardson...|     CL|FIN|
|     mervin.logan|mervin.logan@emai...|     AE|FIN|
|  sherell.alvarez|sherell.alvarez@e...|     EG|FIN|
| valentine.porter|valentine.porter@...|     ID|FIN|
|  nicholle.holden|nicholle.holden@e...|     I

In [0]:
CC_Rem = CC_Rem.withColumn('Comment', lit('Remove these CN users from CC grp'))
DC_Rem = DC_Rem.withColumn('Comment', lit('Remove these CN users from DC grp'))
CC_CN_Rem = CC_CN_Rem.withColumn('Comment', lit('Remove these other region users from CC_CN grp'))
DC_CN_Rem = DC_CN_Rem.withColumn('Comment', lit('Remove these other region users from DC_CN grp'))

In [0]:
def merge_data(df,dlist):
    for i in range(len(dlist)):
        df = df.union(dlist[i])
    return df

In [0]:
data_anamolies = merge_data(CC_Rem,[DC_Rem,CC_CN_Rem,DC_CN_Rem])

In [0]:
data_anamolies.count()

Out[34]: 8

In [0]:
df = data_anamolies.toPandas()

In [0]:
df.to_csv("/dbfs/mnt/rawdata/dest/Data_Anamolies.csv", index = False)